In [1]:
# ✅ 최신 패키지 설치 (transformers 포함)
!pip install --upgrade --no-cache-dir transformers datasets evaluate librosa jiwer accelerate --quiet

In [2]:
from huggingface_hub import login
login()

In [3]:
#데이터셋 로딩(한국어 버전)
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_13_0", "ko", split="train")
print(f"샘플 수: {len(dataset)}")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


샘플 수: 192


In [4]:
#Whisper 전처리기 설정(small 모델 사용)
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

checkpoint = "openai/whisper-small"

feature_extractor = WhisperFeatureExtractor.from_pretrained(checkpoint)
tokenizer = WhisperTokenizer.from_pretrained(checkpoint, language="ko", task="transcribe")
processor = WhisperProcessor.from_pretrained(checkpoint, language="ko", task="transcribe")

In [5]:
#전처리 함수 정의 및 적용
def prepare_dataset(batch):
    # 1. 오디오 → 입력 특징 추출
    audio = batch["audio"]["array"]
    batch["input_features"] = feature_extractor(audio, sampling_rate=16000).input_features[0]

    # 2. 텍스트 → 라벨 토큰 ID
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

processed_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

In [6]:
#모델 로드
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(checkpoint)

In [15]:
#Whisper 전용 collator정의
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorWhisperWithPadding:
    processor: Any
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch


In [16]:
#collator 인스턴스 생성
data_collator = DataCollatorWhisperWithPadding(processor=processor)

In [17]:
#TrainingArguments 설정
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./whisper-small-ko",     # 결과 저장 경로
    per_device_train_batch_size=4,       # GPU: 4 / CPU: 2
    learning_rate=1e-5,
    num_train_epochs=3,
    logging_steps=5,
    save_strategy="no",
    fp16=True,                           # GPU만 True
    report_to="none"
)

In [18]:
#Trainer 구성
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    tokenizer=processor,
    data_collator=data_collator
)

<ipython-input-18-456a5ce04e1f>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
#모델 학습 시작
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
5,5.936700
10,5.387200
15,4.546900
20,3.995100
25,3.902400
30,3.352400
35,2.577600
40,2.237500
45,2.020400
50,1.721800


TrainOutput(global_step=144, training_loss=1.807467473877801, metrics={'train_runtime': 148.1444, 'train_samples_per_second': 3.888, 'train_steps_per_second': 0.972, 'total_flos': 1.6622519058432e+17, 'train_loss': 1.807467473877801, 'epoch': 3.0})

- global_step=144,#총 144step 학습함(즉, 배치 144번)
- training_loss=1.8074674739877801,
- 'train_runtime': 148.1444, #전체 학습 시간
- 'train_samples_per_second': 3.888, #초당 처리된 샘플 수
- 'train_steps_per_second': 0.972, #초당 학습 스텝 수
- 'total_flos': 1.6622519058432e+17,#부동소수점 연산량, 성능보다는 학습 규모 추정용임
- 'train_loss': 1.8074674739877801, #학습이 끝났을 때 모델이 예측한 값과 실제 정답 간의 평균 오차값
- 'epoch': 3.0 #전체 데이터를 3번 반복


In [20]:
import numpy as np
from tqdm import tqdm
from jiwer import wer

# 예측 결과 저장용 리스트
predictions = []
references = []

# 전체 데이터셋에 대해 예측 수행
for sample in tqdm(dataset):
    input_features = processor.feature_extractor(
        sample["audio"]["array"], sampling_rate=16000, return_tensors="pt"
    ).input_features.to(model.device)

    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    predictions.append(transcription)
    references.append(sample["sentence"])

# 정확도 평가
wer_score = wer(references, predictions)
print(f"\n✅ 전체 WER (Word Error Rate): {wer_score:.4f}")

# 예측 결과 하나씩 출력
print("\n📋 예측 결과 전체 비교:")
for i in range(len(predictions)):
    print(f"\n🔹 예측 {i+1}: {predictions[i]}")
    print(f"🔸 정답 {i+1}: {references[i]}")

  0%|          | 0/192 [00:00<?, ?it/s]Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 192/192 [11:52<00:00,  3.71s/it]


✅ 전체 WER (Word Error Rate): 1.2526

📋 예측 결과 전체 비교:

🔹 예측 1: 
🔸 정답 1: 어느덧 그 더운 팔월도 하루를 남기고 다 지나 버렸다.

🔹 예측 2:  ឍ្្្្្្្្្្្្្្្្្្្្្្្្
🔸 정답 2: 근자에 춘우의 동료 사이에는 이상한 소문이 돌아다니었다.

🔹 예측 3:  ឍ្្្្្្្្្្្្្្្្្្្្្្្្្
🔸 정답 3: 재차 부르는 소리를 듣고야 선비는 발길을 떼었다.

🔹 예측 4: 그의 발갛을 찾아나 봤으면 되는 것 아니냐.
🔸 정답 4: 그애의 말값으로 찾아나 봤으면 되는 것 아니냐.

🔹 예측 5: 그가 방으로 들어오니 간난이가 와서 그의 하던 일을 하고 있었다.
🔸 정답 5: 그가 방으로 들어오니 간난이가 와서 그의 하던 일을 하고 있었다.

🔹 예측 6:  ឍ្្្្្
🔸 정답 6: 안녕히 다녀옵시요.

🔹 예측 7:  ឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡឡ
🔸 정답 7: 어떻게 무엇부터 시작을 해야 할는지 엄두가 나지를 않아서 잠을 잘 수가 없었다.

🔹 예측 8:  ឍ្្្្្្្្្្្្្្្្្្
🔸 정답 8: 죽는 날까지 하늘을 우러러

🔹 예측 9:  ឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍឍ
🔸 정답 9: 그리고 그 어두운 방 안이 휙 지나친다.

🔹 예측 10: ತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತಿತತಿತತಿತಿತತಿತಿತತಿತತಿತತಿತತಿತತಿತತಿತತಿತತಿತತಿತತಿತತಿತತಿತತತಿತತತಿತತತಿತತತತತತತತತತತತತತತತತತತತತತತತತ